In [33]:
# NFL Game Outcome Interaction Model with Rolling Window Training and Voila Visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from ipywidgets import widgets
from IPython.display import display, clear_output
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px

In [38]:
# ---------------------------
# Helper Functions
# ---------------------------
def get_previous_week(current_week, all_weeks_sorted):
    season = current_week // 100
    week = current_week % 100
    if week == 1:
        prev_week = (season - 1) * 100 + 18
    else:
        prev_week = season * 100 + (week - 1)
    return prev_week if prev_week in all_weeks_sorted else None

def build_interaction_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [39]:
# ---------------------------
# Load Schedule and Embeddings
# ---------------------------
gru_embedding_df = pd.read_csv('gru_team_embeddings_by_week.csv')
mlp_embedding_df = pd.read_csv('mlp_2023_2024_embeddings.csv')
mlp_embedding_df.set_index('team', inplace=False)

# Embedding column separation
gru_embedding_cols = [col for col in gru_embedding_df.columns if col.startswith('emb_')]
mlp_embedding_cols = [col for col in mlp_embedding_df.columns if col.startswith('mlp_emb_')]

# Week list
all_weeks = sorted(gru_embedding_df['seasonweek'].unique())
upcoming_weeks = [202501, 202502, 202503]
full_week_list = sorted(set(all_weeks + upcoming_weeks))

In [43]:
# ---------------------------
# Model & Prediction
# ---------------------------
interaction_model = load_model('interaction_model_rolling_window.keras')

def predict_game(team_a, team_b, week, model, mlp_df, gru_df, gru_cols, mlp_cols):
    prev_week = get_previous_week(week, all_weeks)
    try:
        t1_gru = gru_df.loc[(gru_df['team'] == team_a) & (gru_df['seasonweek'] == prev_week), gru_cols].values.flatten()
        t2_gru = gru_df.loc[(gru_df['team'] == team_b) & (gru_df['seasonweek'] == prev_week), gru_cols].values.flatten()

        t1_mlp = mlp_df.loc[mlp_df['team'] == team_a, mlp_cols].values.flatten()
        t2_mlp = mlp_df.loc[mlp_df['team'] == team_b, mlp_cols].values.flatten()
    except Exception as e:
        return f"❌ Embedding error: {e}", None

    input_vector = np.concatenate([t1_gru, t1_mlp, t2_gru, t2_mlp]).reshape(1, -1)
    try:
        prob = model.predict(input_vector)[0][0]
        winner = team_a if prob >= 0.5 else team_b
        return f"✅ {team_a} win probability: {prob:.4f}\n🏆 Predicted winner: {winner}", prob
    except Exception as e:
        return f"❌ Prediction error: {e}", None

In [44]:
# ---------------------------
# Widgets
# ---------------------------
team_a_widget = widgets.Dropdown(
    options=mlp_embedding_df['team'].tolist(), description="Team A:"
)
team_b_widget = widgets.Dropdown(
    options=mlp_embedding_df['team'].tolist(), description="Team B:"
)
week_widget = widgets.Dropdown(
    options=[(f"{w // 100} Week {w % 100}", w) for w in full_week_list],
    value=full_week_list[0],
    description="Week:"
)

output = widgets.Output()
loadings_output = widgets.Output()
pca_loadings_by_week = {}

In [45]:
# ---------------------------
# Animation with PCA loadings tracking
# ---------------------------
def animate_gru_embeddings(gru_df):
    team_a = team_a_widget.value
    team_b = team_b_widget.value

    all_pca_rows = []
    weeks = sorted(gru_df['seasonweek'].unique())

    for week in weeks:
        df_week = gru_df[gru_df['seasonweek'] == week]
        if df_week.empty:
            continue
        emb = df_week[gru_embedding_cols].values
        pca = PCA(n_components=2)
        coords = pca.fit_transform(emb)

        # Save loadings
        pca_loadings_by_week[week] = pca.components_

        for j, (_, row) in enumerate(df_week.iterrows()):
            team = row['team']
            all_pca_rows.append({
                'team': team,
                'PC1': coords[j, 0],
                'PC2': coords[j, 1],
                'week': week,
                'highlight': 'Team A' if team == team_a else ('Team B' if team == team_b else 'Other')
            })

    pca_df = pd.DataFrame(all_pca_rows)
    fig = px.scatter(
        pca_df, x='PC1', y='PC2', animation_frame='week', color='highlight',
        hover_name='team',
        title="GRU Embedding PCA Animation",
        category_orders={'week': sorted(pca_df['week'].unique())},
        height=600
    )
    fig.update_traces(marker=dict(size=10, opacity=0.8))
    fig.update_layout(showlegend=True)
    return fig

In [46]:
# ---------------------------
# Embedding Visualizations
# ---------------------------
def generate_embedding_map(gru_df, selected_teams, target_week):
    latest_gru = gru_df[gru_df['seasonweek'] == target_week].copy()
    if latest_gru.empty:
        print(f"No GRU embeddings found for week {target_week}.")
        return

    emb_only = latest_gru[gru_embedding_cols].values
    teams = latest_gru['team'].values

    pca = PCA(n_components=2)
    coords = pca.fit_transform(emb_only)

    plt.figure(figsize=(8, 6))
    for i, team in enumerate(teams):
        color = 'red' if team in selected_teams else 'blue'
        plt.scatter(coords[i, 0], coords[i, 1], color=color)
        plt.text(coords[i, 0] + 0.1, coords[i, 1], team, fontsize=9)
    plt.title(f"GRU Team Embeddings (PCA) - Week {target_week}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def compare_embeddings(team_a, team_b, mlp_df, gru_df):
    try:
        mlp_a = mlp_df.loc[mlp_df['team'] == team_a, mlp_embedding_cols].values.flatten()
        mlp_b = mlp_df.loc[mlp_df['team'] == team_b, mlp_embedding_cols].values.flatten()
        gru_a = gru_df.loc[gru_df['team'] == team_a].iloc[-1][gru_embedding_cols].values.flatten()
        gru_b = gru_df.loc[gru_df['team'] == team_b].iloc[-1][gru_embedding_cols].values.flatten()

        sim_mlp = cosine_similarity([mlp_a], [mlp_b])[0][0]
        sim_gru = cosine_similarity([gru_a], [gru_b])[0][0]

        plt.barh(['GRU Similarity', 'MLP Similarity'], [sim_gru, sim_mlp], color=['green', 'orange'])
        plt.title("Team Embedding Similarity")
        plt.xlim(0, 1)
        plt.show()
    except Exception as e:
        print(f"Similarity comparison failed: {e}")

In [47]:
# ---------------------------
# PCA loadings display
# ---------------------------
def update_pca_loadings_display(selected_week):
    with loadings_output:
        clear_output()
        components = pca_loadings_by_week.get(selected_week)
        if components is not None:
            print(f"\n🔥 Top PCA Loadings for Week {selected_week}:")
            for i, comp in enumerate(components):
                axis = f"PC{i + 1}"
                top_dims = sorted([(j, abs(w)) for j, w in enumerate(comp)], key=lambda x: -x[1])[:5]
                print(f"{axis}:")
                for dim_idx, _ in top_dims:
                    weight = comp[dim_idx]
                    marker = "🔥" if abs(weight) > 0.3 else ""
                    print(f"  {marker} emb_{dim_idx:<2}: {weight:.3f}")
        else:
            print("No PCA loadings available for this week.")

week_widget.observe(lambda change: update_pca_loadings_display(change['new']), names='value')

In [49]:
# ---------------------------
# Button Handler
# ---------------------------
def on_button_click(b):
    with output:
        clear_output()
        team_a = team_a_widget.value
        team_b = team_b_widget.value
        week = week_widget.value

        msg, _ = predict_game(team_a, team_b, week, interaction_model,
                              mlp_embedding_df, gru_embedding_df,
                              gru_embedding_cols, mlp_embedding_cols)
        print(msg)
        compare_embeddings(team_a, team_b, mlp_embedding_df, gru_embedding_df)
        prev_week = get_previous_week(week, all_weeks)
        #generate_embedding_map(gru_embedding_df, [team_a, team_b], prev_week)
        fig = animate_gru_embeddings(gru_embedding_df)
        display(fig)
        update_pca_loadings_display(week)

# ---------------------------
# Display widgets and animation
# ---------------------------
predict_button = widgets.Button(description="Predict Game")
predict_button.on_click(on_button_click)

display(team_a_widget, team_b_widget, week_widget, predict_button)
display(output)
display(loadings_output)

Dropdown(description='Team A:', index=13, options=('CRD', 'ATL', 'RAV', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DA…

Dropdown(description='Team B:', index=11, options=('CRD', 'ATL', 'RAV', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DA…

Dropdown(description='Week:', index=36, options=(('2022 Week 2', np.int64(202202)), ('2022 Week 3', np.int64(2…

Button(description='Predict Game', style=ButtonStyle())

Output(outputs=({'name': 'stdout', 'text': '\x1b1/1\x1b \x1b━━━━━━━━━━━━━━━━━━━━\x1b\x1b\x1b[…

Output(outputs=({'name': 'stdout', 'text': 'No PCA loadings available for this week.\n', 'output_type': 'strea…